### Hybrid Search Langchain

In [2]:
%pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [1]:
from dotenv import load_dotenv
import os

In [11]:
load_dotenv()

True

In [12]:
api_key = os.getenv("PINECONE_API_KEY")

In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
from pinecone import Pinecone, ServerlessSpec

c:\Users\Divy Solanki\.conda\envs\llm\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
index_name = "hybrid-search-langchain-pinecone"

# initialize the pinecone client
pc = Pinecone(api_key=api_key)

#create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of the dense vector
        metric="dotproduct", # sparse value supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [7]:
index = pc.Index(index_name)
index

In [13]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [14]:
# vector embeddings and sparse matrix

from langchain_huggingface import HuggingFaceEmbeddings # type: ignore
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-V2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-V2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [16]:
sentences = [
    "In the 2023, I visited Paris",
    "In the 2022, I visited New York",
    "In the 2021, I visited New Orleans"
]

# TF-IDF values on these sentences
bm25_encoder.fit(sentences)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load the BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 75.27it/s]


In [17]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [18]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-V2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x00000218E7B5C110>, index=<pinecone.data.index.Index object at 0x00000218BC3D4A90>)

In [20]:
retriever.add_texts(
    [
        "In the 2023, I visited Paris",
        "In the 2022, I visited New York",
        "In the 2021, I visited New Orleans"
    ]
)

100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


In [22]:
retriever.invoke("What city did I visited recently?")

[Document(page_content='In the 2021, I visited New Orleans'),
 Document(page_content='In the 2022, I visited New York'),
 Document(page_content='In the 2023, I visited Paris')]

In [23]:
retriever.invoke("What city did I visited in 2022?")

[Document(page_content='In the 2022, I visited New York'),
 Document(page_content='In the 2023, I visited Paris'),
 Document(page_content='In the 2021, I visited New Orleans')]